<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA Tutorial

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

In [ ]:
import warnings

warnings.filterwarnings(action="ignore")

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD.

In [ ]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"

# Remove work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)

# Define files
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "oad_process.yml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "beechcraft_76.xml")

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

After defining a configuration file name, we can ask FAST-GA to generate a default configuration file based on the default OAD model implemented in the framework:

In [ ]:
api_cs25.generate_configuration_file(
    CONFIGURATION_FILE,
    overwrite=True,
    distribution_name="fast-oad-cs23",
    sample_file_name="fastga.yml",
)

You can now checkout the generated [configuration file](./workdir/oad_process.yml). In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [ ]:
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [ ]:
api_cs25.list_variables(CONFIGURATION_FILE)

Another useful feature is to list the modules of the model defined in the configuration file:

In [ ]:
api_cs25.list_modules(CONFIGURATION_FILE)

Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
api_cs25.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame

IFrame(src=N2_FILE, width="100%", height="500px")

Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [ ]:
XDSM_FILE = pth.join(WORK_FOLDER_PATH, "xdsm.html")
api_cs25.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
from IPython.display import IFrame

IFrame(src=XDSM_FILE, width="100%", height="500px")

## 2. Running your first MDA

### Beechcraft
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [ ]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
Beechcraft_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs_Beechcraft_800nm_mda.xml")
shutil.copy(OUTPUT_FILE, Beechcraft_OUTPUT_FILE)

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [ ]:
api_cs25.variable_viewer(OUTPUT_FILE)

In particular, you may inspect the `data:handling_qualities:static_margin` variable.
You will see that its value is equal to the value we set as the target meaning the MDA process converged correctly.

### Beechcraft for 1000 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 1000 nm range instead of 800 nm for the Beechcraft. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [ ]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_inputs.xml")
api_cs25.variable_viewer(INPUT_FILE)

Now that the range has been changed, we run again the MDA.

In [ ]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

Let's save again these new results, for post-processing them in next notebook.

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
Beechcraft_1000nm_OUTPUT_FILE = pth.join(
    WORK_FOLDER_PATH, "problem_outputs_Beechcraft_1000nm_mda.xml"
)
shutil.copy(OUTPUT_FILE, Beechcraft_1000nm_OUTPUT_FILE)

## 3. Running your first MDO

## Beechcraft 800 nm optimization

As seen earlier, the current aircraft configuration is estimated with a fixed distance between the wing aerodynamic center and the horizontal tail aerodynamic center. We will unlock this parameter using a simple optimization problem that will move horizontal tail position in order to minimize the mission consumption. 

The default configuration file defines this optimization problem that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the distance between the wing and the tail aerodynamic center (design variables): by extension the horizontal tail to wing distance,
- subject to no additional constraint.

However, running the MDO with the propeller performance estimation will take a very long time. Consequently, and since the propeller won't change from one MDO loop to another, we decided to use a different .yml file for the probleme definition and reuse the propeller performance map computed in the MDA's hence why the input file will be the output file of a previous MDA.

*(This run should take approximately 20 minutes)*

In [ ]:
CONFIGURATION_FILE_MDO = pth.join(WORK_FOLDER_PATH, "oad_process_mdo.yml")
SOURCE_FILE_MDO = pth.join(WORK_FOLDER_PATH, "problem_outputs_Beechcraft_800nm_mda.xml")
shutil.copy(pth.join(DATA_FOLDER_PATH, "fastga_mdo.yml"), CONFIGURATION_FILE_MDO)

In [ ]:
# Set back the inputs from the reference Beechcraft 800 nm
api_cs25.generate_inputs(CONFIGURATION_FILE_MDO, SOURCE_FILE_MDO, overwrite=True)

To visualize and edit the optimization problem definition (present in the configuration file .toml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .toml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [ ]:
api_cs25.optimization_viewer(CONFIGURATION_FILE_MDO)

In [ ]:
optim_problem = api_cs25.optimize_problem(CONFIGURATION_FILE_MDO, overwrite=True)

Let's save these results:

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs_Beechcraft_800nm_mdo.xml")
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [ ]:
api_cs25.optimization_viewer(CONFIGURATION_FILE_MDO)

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [ ]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
api_cs25.variable_viewer(RESULT_FILE)